In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/iterative-tensor-decomposition-colab

/content/drive/MyDrive/iterative-tensor-decomposition-colab


In [3]:
import numpy as np
import scipy.ndimage as spnd
import torch
from run.run_tca import decompose
import pickle


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

d1 = np.load('./data/neurons_10_straight_Churchland.npy').transpose([1,0,2]) * 100
d2 = np.load('./data/neurons_10_curved_Churchland.npy').transpose([1,0,2]) * 100
data = np.concatenate([d1,d2], axis=1)

data = spnd.gaussian_filter1d(data, sigma=2)
data = np.array([d-np.min(d) for d in data])
data = np.array([d/np.max(d) for d in data])
data = torch.from_numpy(data.transpose([1,0,2])).float().to(device)

In [6]:
print(data.shape)
print((1*1*15*188000)/(511*182*150)) # percentage: block mask dims, how many blocks

torch.Size([511, 182, 150])
0.20214619040450743


In [7]:
#The hyperparameters
slice_TCA = True
positive = True
orthogonal_constraint  = False
cross_validation = True
orthogonal_penalty = False
metric = False
do_varimax = False
cut_cv_mask = 4

mask_cross_validation = 0.2

learning_rate = 0.02
mask = 0.8

decay_rate_lr = 1
decay_rate_mask = 0.5
decay_iterations = 3
decay_type_lr='exponential'
decay_type_mask='exponential'

iterations = decay_iterations*149
batch_size = 20
test_freq = -1
initialization = 'uniform-positive'
loss_kernel_sigmas = None

seed = 999

verbose_train = True
verbose_test = True

In [8]:
mask_cross_validation = [0]

res = []
for seed in range(10):
    res.append(decompose(seed=seed, data=data, number_components=[12,0,1], iterations=iterations,
                decay_rate_mask=decay_rate_mask, decay_rate_lr=decay_rate_lr, decay_iterations=decay_iterations,
                decay_type_mask=decay_type_mask, decay_type_lr=decay_type_lr, learning_rate=learning_rate, mask=mask,
                mask_cross_validation=mask_cross_validation[0], batch_size=batch_size, sliceTCA=slice_TCA, positive=positive,
                orthogonal_penalty=orthogonal_penalty, metric=metric, cross_validation=False,
                precision=torch.float32, orthogonal_constraint=orthogonal_constraint, cut_cv_mask=cut_cv_mask,
                initialization=initialization, orthogonal_skip=(), test_freq=-1, verbose_train=verbose_train,
                verbose_test=verbose_test, device=device))

Iteration: 0 	mse_loss: 3.0465669632
Test -- Iteration: 0 	mse_loss: 2.8168802261
Iteration: 1 	mse_loss: 2.8171935081
Iteration: 2 	mse_loss: 2.6022610664
Iteration: 3 	mse_loss: 2.4020912647
Iteration: 4 	mse_loss: 2.2172360420
Iteration: 5 	mse_loss: 2.0471653938
Iteration: 6 	mse_loss: 1.8899700642
Iteration: 7 	mse_loss: 1.7455542088
Iteration: 8 	mse_loss: 1.6134904623
Iteration: 9 	mse_loss: 1.4928222895
Iteration: 10 	mse_loss: 1.3823325634
Iteration: 11 	mse_loss: 1.2814027071
Iteration: 12 	mse_loss: 1.1898528337
Iteration: 13 	mse_loss: 1.1066349745
Iteration: 14 	mse_loss: 1.0308688879
Iteration: 15 	mse_loss: 0.9621300697
Iteration: 16 	mse_loss: 0.8999217749
Iteration: 17 	mse_loss: 0.8434091210
Iteration: 18 	mse_loss: 0.7918075323
Iteration: 19 	mse_loss: 0.7446738482
Iteration: 20 	mse_loss: 0.7017400861
Test -- Iteration: 20 	mse_loss: 0.6623878479
Iteration: 21 	mse_loss: 0.6623658538
Iteration: 22 	mse_loss: 0.6263423562
Iteration: 23 	mse_loss: 0.5932705402
Iterati

KeyboardInterrupt: 

In [ ]:
bestmodel = np.argmin([r[2] for r in res])
components, metric_value, ls, l_cv, convergence, model = res[bestmodel]
pickle.dump(model.cpu(), open('./Churchland_sliceTCA-12-0-1.p', 'wb'))

In [ ]:
models = [r[-1].cpu() for r in res]
losses = [r[2] for r in res]

In [ ]:
pickle.dump([models, losses], open('./Churchland_sliceTCA_12-0-1_all-models.p', 'wb'))